In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import copy

In [2]:
# class Net(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.dense1 = nn.Linear(2,5)
#         self.dense2 = nn.Linear(5,5)
#         self.dense3 = nn.Linear(5,1)
        
#     def forward(self, x):
#         x = self.dense1(x)
#         x = F.relu(x)
#         x = self.dense2(x)
#         x = F.relu(x)
#         x = self.dense3(x)
#         x = self.sigmoid(x)
#         return x

# criterion = nn.MSELoss()
# net = Net()
# print(list(net.parameters()))
# optimizer = optim.SGD(net.parameters(),lr=0.01)

# x = net.parameters().__next__()

# print("{}".format(x.data))
# print("{}".format(x.grad))

# input = torch.tensor(torch.rand(1,2), requires_grad=True)
# print(input)

In [36]:
# forward prop
## sigmoid activation function using pytorch
def sigmoid_activation(z):
    return 1 / (1 + torch.exp(-z))
## activation of hidden layer 
# z1 = torch.mm(x, w1) + b1
# a1 = sigmoid_activation(z1)
# ## activation (output) of final layer 
# z2 = torch.mm(a1, w2) + b2
# output = sigmoid_activation(z2)
# print(x)
# print(z1)
# print(output)


def forward_prop(_data, _nodes, _weights, _biases):
    layer_vals = []
    a = _data
    for i in range(len(_nodes)-2): # feed thru hidden layers
#         print(_weights[i])
#         print(_biases[i])
        z = torch.mm(a, _weights[i]) + _biases[i]
        a = sigmoid_activation(z)
        layer_vals.append(a)
#         print("a after ", i, ":",a)
    # feed thru last layer
#     print("uh")
#     print(_weights[-1], _biases[-1])
#     print(torch.mm(a, _weights[-1]))
    z_out = torch.mm(a, _weights[-1]) + _biases[-1]
    output = sigmoid_activation(z_out)
    layer_vals.append(output)
#     print(output)
    return layer_vals

print("forward prop:\n")
y_out = forward_prop(x, nodes, weights, biases)
print("layer vals", y_out)

forward prop:

layer vals [tensor([[0.6625, 0.8657, 0.0725]]), tensor([[0.7118, 0.5734]]), tensor([[0.2617]])]


In [37]:
# loss computation
# loss = y - output
def calculate_loss(_layer_vals):
    return y - _layer_vals[-1]

In [38]:
# back propogation
## function to calculate the derivative of activation
def sigmoid_delta(x):
  return x * (1 - x)
## compute derivative of error terms
# delta_output = sigmoid_delta(output)
# delta_hidden = sigmoid_delta(a1)

# ## backpass the changes to previous layers 
# d_outp = loss * delta_output
# loss_h = torch.mm(d_outp, w2.t())
# d_hidn = loss_h * delta_hidden

# # update parameters
# learning_rate = 0.1

# w2 += torch.mm(a1.t(), d_outp) * learning_rate
# w1 += torch.mm(x.t(), d_hidn) * learning_rate

# b2 += d_outp.sum() * learning_rate
# b1 += d_hidn.sum() * learning_rate

# print(w2, w1)

In [39]:
print(weights)

[tensor([[ 0.1363,  0.9971,  0.0078],
        [ 0.4750,  0.3185,  1.3495],
        [-0.4299,  0.6760, -0.4416],
        [-0.5805, -0.6495,  0.3358],
        [-1.0582, -2.1176,  1.2568]]), tensor([[ 0.6560, -0.8607],
        [ 0.4205, -0.8615],
        [ 0.5707,  0.7148]]), tensor([[-1.1920],
        [ 0.0596]])]


In [40]:
print(y_out)

def backprop_and_update(_layer_vals, _weights, _biases, _data):
    loss = calculate_loss(_layer_vals)
    deltas = [] # from first hidden to output
    ds = [] # from output to first hidden
    # compute derivative of error terms
    for a in _layer_vals:
        deltas.append(sigmoid_delta(a))
    loss_d = loss
    for d, w in zip(reversed(deltas), reversed(_weights)):
        dd = loss * d
        loss_d = torch.mm(d, w.t())
        ds.append(dd)
    learning_rate = 0.1
    _as = copy.deepcopy(_layer_vals)
    _as.insert(0,x)
    del _as[-1]
#     print("\n Layer vals \n", _layer_vals)
    new_weights = []
    new_biases = []
    print("lengths", len(ds), len(_weights), len(_as))
    for d, w, a, b in zip(ds, reversed(_weights), _as, reversed(_biases)):
        wt = torch.mm(a.t(),d) * learning_rate
        new_weights.insert(0, wt)
        
        bi = b + d.sum()*learning_rate
        new_biases.insert(0,bi)
    return new_weights,  new_biases   

[tensor([[0.6625, 0.8657, 0.0725]]), tensor([[0.7118, 0.5734]]), tensor([[0.2617]])]


In [41]:
print(weights)
weights_2, biases_2 = backprop_and_update(y_out, weights, biases, x)
print("Weights",weights_2 )

[tensor([[ 0.1363,  0.9971,  0.0078],
        [ 0.4750,  0.3185,  1.3495],
        [-0.4299,  0.6760, -0.4416],
        [-0.5805, -0.6495,  0.3358],
        [-1.0582, -2.1176,  1.2568]]), tensor([[ 0.6560, -0.8607],
        [ 0.4205, -0.8615],
        [ 0.5707,  0.7148]]), tensor([[-1.1920],
        [ 0.0596]])]
lengths 3 3 3
Weights [tensor([[-0.0055, -0.0029, -0.0017],
        [-0.0044, -0.0023, -0.0013]]), tensor([[-0.0047, -0.0056],
        [-0.0061, -0.0073],
        [-0.0005, -0.0006]]), tensor([[ 0.0033],
        [ 0.0026],
        [-0.0028],
        [-0.0038],
        [ 0.0106]])]


In [42]:
def train(_data, _labels, _nodes, _epochs):
    #param initialization
    weights = []
    for i in range(len(_nodes)-1):
        w = torch.randn(_nodes[i], _nodes[i+1])
        weights.append(w)
    biases = []
    for i in range(len(nodes)-1):
        b = torch.randn(1, nodes[i+1])
        biases.append(b)
    
    print("WEIGHTS ARRAY:\n", weights)
    print("BIASES ARRAY:\n", biases)

    print("\n TRAINING \n")
    y_out = forward_prop(_data, _nodes, weights, biases)
    weights_2, biases_2 = backprop_and_update(y_out, weights, biases, _data)
    print("Final Weights: \n", weights_2)
    print("Final Biases: \n", biases_2)
    
   

In [43]:
## initialize tensor for inputs, and outputs 
x = torch.randn((1, 5))
y = torch.randn((1, 1))
nodes = [5, 3, 2, 1]
train(x, y, nodes, 10)

WEIGHTS ARRAY:
 [tensor([[ 1.2508, -0.8454,  0.4690],
        [-1.1219, -0.5904,  0.1739],
        [ 1.7703,  0.9717,  1.6507],
        [-0.3860,  0.8910,  2.8173],
        [-0.3859,  1.3775, -1.0677]]), tensor([[ 1.8160, -0.0628],
        [-0.1015,  0.8879],
        [-0.6252,  0.2118]]), tensor([[ 0.0629],
        [-0.8303]])]
BIASES ARRAY:
 [tensor([[-0.9305, -0.9786, -0.2937]]), tensor([[-1.4985,  0.6054]]), tensor([[-0.6038]])]

 TRAINING 

lengths 3 3 3
Final Weights: 
 [tensor([[-0.0012, -0.0033, -0.0002],
        [-0.0043, -0.0117, -0.0007]]), tensor([[-0.0011, -0.0014],
        [-0.0038, -0.0049],
        [-0.0002, -0.0002]]), tensor([[ 0.0013],
        [ 0.0111],
        [ 0.0166],
        [ 0.0055],
        [-0.0110]])]
Final Biases: 
 [tensor([[-0.9543, -1.0024, -0.3175]]), tensor([[-1.5277,  0.5762]]), tensor([[-0.6181]])]
